Scoring training data for miniML. After the plot has been opened, it can be closed at any time and e.g. go to a different starting index. As long as the kernel does not crash, the changes in the scores will be retained and can just be saved at the end.

**Applied changes**

first round of scoring for nice events (1) and false positives (0), everything else is left in blue
second round of scoring - not_so_nice events (1) and noise (0)
saved for each file individually in a desginated folder
after that each file needs to be moved (manually for now) to the everything_scored folder
- otherwise the indexing in the second code cell won't work

script to go thourt all and count
script to collect all all categories and run though them once again
script to collected all the category separated files into one complete dataset

In [2]:
import h5py
import os
import matplotlib.pyplot as plt
from ScoringPanel import ScoringPanel
import numpy as np

plt.style.use('default')
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'

In [2]:
output_folder = '/Users/verjim/miniML_multipatch/model_training/extract_training_data/output/Oct_2024_train/'
all_files = os.listdir(output_folder)
h5s_ = sorted([x for x in all_files if x[-3:] == '.h5'])

In [ ]:
j = 0

i = h5s_[j]
# Load data to be scored
file_path = output_folder + i
with h5py.File(file_path, 'r') as f:
    x = f['events'][:]
    y = f['scores'][:]

print(i)
print(x.shape, y.shape)

Open the interactive plot for scoring.
- To go to the next/previous event, use forward/backward arrows on the keyboard or the scroll wheel.
- To change the score, press 'm' on the keyboard.


The scores mean the following:
- 0: red, not an event of interest
- 1: black, event of interest
- 2: blue, unclear

For training the model, there should be no events with scores 2 in the dataset. We added this so one can mark them and either exclude them or come back to them at a later time point.

In [4]:
%matplotlib qt
# Open plot for scoring
fig, ax = plt.subplots(1, 1)
tracker = ScoringPanel(fig, ax, x, y, start_ind=0)
fig.canvas.mpl_connect('key_press_event', tracker.onclick)
fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
plt.show()


In [6]:
false_positives = x[np.where(y==0)].copy()
false_positives_score = y[y == 0].copy()

okey_events = x[np.where(y==1)].copy()
okey_events_score = y[y == 1].copy()

# nice_events = x[np.where(y==1)].copy()
# nice_events_score = y[y == 1].copy()

# unscored_x = x[np.where(y==2)].copy()  
# unscored = y[y == 2].copy()

# if len(false_positives) + len(nice_events) + len(unscored) == len(y):
#     print('free to continue')

In [7]:
### Save modified dataset. Changes will only be saved after running this cell.

with h5py.File(output_folder + 'false_positives/' +  i[:-3] + 'false_pos.h5', 'w') as f:
    f.create_dataset("events", data = false_positives)
    f.create_dataset("scores", data = false_positives_score)


with h5py.File(output_folder + 'not_so_nice_events/' + i[:-3] + '_okey.h5', 'w') as f:
    f.create_dataset("events", data = okey_events)
    f.create_dataset("scores", data = okey_events_score)

# with h5py.File(output_folder + 'nice_events/' + i[:-3] + '_nice.h5', 'w') as f:
#     f.create_dataset("events", data = nice_events)
#     f.create_dataset("scores", data = nice_events_score)

# with h5py.File(output_folder + 'unscored_round_1/' + i[:-3] + '_unscored.h5', 'w') as f:
#     f.create_dataset("events", data = unscored_x)
#     f.create_dataset("scores", data = unscored)

From the previously unscored, extract not so nice/ clear events and noise

The scores mean the following:
- 0: red, noise
- 1: black, not so nice event
- 2: blue, unclear, delete

In [7]:
%matplotlib qt
# Open plot for scoring
fig, ax = plt.subplots(1, 1)
tracker = ScoringPanel(fig, ax, unscored_x, unscored, start_ind=0)
fig.canvas.mpl_connect('key_press_event', tracker.onclick)
fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
plt.show()

In [ ]:
noise = unscored_x[np.where(unscored==0)].copy()
noise_score = unscored[unscored == 0].copy()
okey_events = unscored_x[np.where(unscored==1)].copy()
okey_events_score = unscored[unscored == 1].copy()

rest_x = unscored_x[np.where(unscored==2)].copy()  
rest = unscored[unscored == 2].copy()

if len(noise) + len(okey_events) + len(rest) == len(unscored):
    print('free to continue')

In [10]:
with h5py.File(output_folder + 'noise/' +  i[:-3] + 'noise.h5', 'w') as f:
    f.create_dataset("events", data = noise)
    f.create_dataset("scores", data = noise_score)

with h5py.File(output_folder + 'not_so_nice_events/' + i[:-3] + '_okey.h5', 'w') as f:
    f.create_dataset("events", data = okey_events)
    f.create_dataset("scores", data = okey_events_score)

In [ ]:
# move the file to the scored folder


# Collecting all the scored

In [10]:
type = 'false_positives/' #not_so_nice_events/, 'false_positives/'

output_folder = '/Users/verjim/miniML_multipatch/model_training/extract_training_data/output/Oct_2024_train/' + type
all_files = os.listdir(output_folder)

# all_files = sorted([x for x in all_files if x[-12:] == 'datanoise.h5'])

In [ ]:
counter = 0
for fn in all_files:
    file_path = output_folder + fn
    with h5py.File(file_path, 'r') as f:
        x = f['events'][:]
        y = f['scores'][:]
    counter += len(x)

print(counter)

# to 12: FP - 564, nice - 308, noise - 93, not_so_nice - 334

## Count for each event type

In [8]:
type = 'nice_events/' # 'not_so_nice_events/', 'false_positives/', 'noise/', 'nice_events/'

scores_dict = {'false_positives/': 0, 'not_so_nice_events/': 1 , 'noise/': 0,'nice_events/': 1}

output_folder = '/Users/verjim/miniML_multipatch/model_training/extract_training_data/output/Oct_2024_train/'
all_files = os.listdir(output_folder + type)

if '.DS_Store' in all_files:
    all_files.remove('.DS_Store')

In [ ]:
for i, file in enumerate(all_files):
    fn = output_folder + type + file
    count = 0
    if i == 0:
        with h5py.File(fn, 'r') as f:
            x = f['events'][:][:, :750]
            y = f['scores'][:]
    else:
        with h5py.File(fn, 'r') as f:
            x = np.concatenate((x, f['events'][:][:, :750]))
            y = np.concatenate((y, f['scores'][:]), axis = None)

print(x.shape, y.shape)

In [10]:
%matplotlib qt
# Open plot for scoring
fig, ax = plt.subplots(1, 1)
tracker = ScoringPanel(fig, ax, x, y, start_ind = 0)
fig.canvas.mpl_connect('key_press_event', tracker.onclick)
fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
plt.show()

In [11]:
score = scores_dict[type]

events_keep = x[np.where(y==score)].copy()
scores_keep = y[y == score].copy()

In [12]:
with h5py.File(output_folder + type[:-1] + '_complete.h5', 'w') as f:
    f.create_dataset("events", data = events_keep)
    f.create_dataset("scores", data = scores_keep)

## Collect all into one dataset

In [25]:
output_folder = '/Users/verjim/miniML_multipatch/model_training/extract_training_data/output/Oct_2024_train/'
all_files = os.listdir(output_folder)

collected = [x for x in all_files if x[-3:] == '.h5']

dict_nums = {'false': 500, 'not_s': 300, 'noise': 100, 'nice_': 300}

In [ ]:
np.shape(x[:500])

In [ ]:
for i, file in enumerate(collected):
    fn = output_folder + file
    count = dict_nums[file[:5]]
    if i == 0:
        with h5py.File(fn, 'r') as f:
            x = f['events'][:count]
            y = f['scores'][:count]
    else:
        with h5py.File(fn, 'r') as f:
            x = np.concatenate((x, f['events'][:count]))
            y = np.concatenate((y, f['scores'][:count]), axis = None)

print(x.shape, y.shape)

In [39]:
with h5py.File(output_folder + 'complete_training_dataset.h5', 'w') as f:
    f.create_dataset("events", data = x)
    f.create_dataset("scores", data = y)